In [1]:
import pandas as pd
import datetime
import random
from collections import defaultdict

# Load data
startups = pd.read_csv("Startups.csv")
mentors = pd.read_csv("Mentor.csv")

# Fill empty fields with "any"
startups.fillna("any", inplace=True)
mentors.fillna("any", inplace=True)

# Initialize time slots and schedules
start_time = datetime.datetime.strptime("11:00", "%H:%M")
end_time = datetime.datetime.strptime("14:00", "%H:%M")
time_slot_duration = datetime.timedelta(minutes=15)
gap_duration = datetime.timedelta(minutes=5)

# Sector-wise distribution of startups
sector_startups = defaultdict(list)
for idx, row in startups.iterrows():
    sector_startups[row['sector']].append(idx)

# Mentor schedule and tracking
mentor_schedule = defaultdict(list)
mentor_startup_tracker = defaultdict(set)  # Tracks which startups each mentor has met
allocated_startups = set()

# Probability distribution function
def get_startup_by_preference(mentor, preferences):
    """
    Select a startup based on the mentor's preferences and probability ratio 5:3:2.
    """
    probabilities = [0.5, 0.3, 0.2]
    for i, sector in enumerate(preferences):
        if sector in sector_startups and sector_startups[sector]:
            if random.random() <= probabilities[i]:  # Check probability
                return sector_startups[sector].pop(0)  # Allocate a startup
    return None  # No startup could be allocated from preferences

# Assign startups to mentors
def assign_startups():
    global allocated_startups
    while True:
        unallocated_startups = set(startups.index) - allocated_startups

        # Break if no unallocated startups are left
        if not unallocated_startups:
            break

        for _, mentor in mentors.iterrows():
            mentor_name = mentor['Name']
            preferences = [mentor['Sector 1'], mentor['Sector 2'], mentor['Sector 3']]

            # Allocate a startup based on preference probabilities
            current_time = start_time if not mentor_schedule[mentor_name] else datetime.datetime.strptime(
                mentor_schedule[mentor_name][-1][0], "%H:%M"
            ) + time_slot_duration + gap_duration
            current_time = max(current_time, start_time)  # Ensure time advances correctly

            if current_time >= end_time:
                return

            startup_idx = get_startup_by_preference(mentor, preferences)
            if startup_idx is None:
                continue

            # Assign time slot and update tracking
            mentor_schedule[mentor_name].append((current_time.strftime("%H:%M"), startup_idx))
            mentor_startup_tracker[mentor_name].add(startup_idx)
            allocated_startups.add(startup_idx)

    # Allow repetition after all startups are assigned at least once
    while True:
        for _, mentor in mentors.iterrows():
            mentor_name = mentor['Name']
            preferences = [mentor['Sector 1'], mentor['Sector 2'], mentor['Sector 3']]

            current_time = start_time if not mentor_schedule[mentor_name] else datetime.datetime.strptime(
                mentor_schedule[mentor_name][-1][0], "%H:%M"
            ) + time_slot_duration + gap_duration

            if current_time >= end_time:
                return

            startup_idx = get_startup_by_preference(mentor, preferences)
            if startup_idx is None or startup_idx in mentor_startup_tracker[mentor_name]:
                continue  # Skip already met startups

            # Assign time slot and update tracking
            mentor_schedule[mentor_name].append((current_time.strftime("%H:%M"), startup_idx))
            mentor_startup_tracker[mentor_name].add(startup_idx)

# Assign startups and display the schedule
assign_startups()

# Display results
for mentor, schedule in mentor_schedule.items():
    print(f"Mentor: {mentor}")
    for time, startup_idx in schedule:
        print(f"  Time: {time}, Startup Index: {startup_idx}, Name: {startups.loc[startup_idx, 'Name']}")
    print("\n")


KeyboardInterrupt: 